In [43]:
import numpy as np

In [48]:
import gzip
vecs = {}
for i, line in enumerate(gzip.open("zh-50k.vec.gz", mode="rt", encoding="utf8")):
    line = line.strip()
    fields = line.split()
    word = fields[0]
    vec = np.array([float(x) for x in fields[1:]])
    if len(vec) != 300:
        continue
    vecs[word] = vec

In [49]:
vecs["猫"]

array([-0.080654,  0.92857 , -0.80243 ,  0.07959 ,  1.0905  , -0.36953 ,
       -0.61218 ,  0.13939 ,  0.58874 ,  0.32197 ,  1.2121  ,  0.25708 ,
        1.2921  , -1.1398  ,  0.41503 , -0.89934 ,  0.80209 , -0.56484 ,
       -1.4821  , -0.80865 , -1.1751  ,  1.4621  ,  1.3497  , -0.055161,
        1.142   , -0.64281 ,  1.6341  , -0.10347 ,  1.0953  ,  0.26906 ,
        0.91051 , -1.1689  , -1.0473  , -0.64934 , -0.68416 ,  1.4666  ,
        1.3768  , -0.46799 ,  0.37927 ,  1.7922  ,  1.1464  ,  1.7355  ,
        1.4968  , -1.2961  , -0.75857 , -0.49246 , -1.0988  , -0.88717 ,
        0.31625 , -1.5837  ,  1.0528  , -0.51194 , -0.50414 , -0.62568 ,
        0.22494 , -1.2283  , -1.1931  , -1.6217  , -4.3753  ,  0.60226 ,
        1.0873  , -0.16854 ,  1.2796  ,  1.1727  ,  0.51944 ,  1.0567  ,
       -1.3084  , -0.72226 , -0.32162 ,  0.18234 ,  1.7002  , -0.69738 ,
        1.5555  , -0.67442 , -1.1027  ,  0.68776 , -0.10535 , -1.5393  ,
       -0.50548 , -1.1877  , -0.54125 ,  0.25082 , 

In [50]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

# cosine similarity
def cosine(v1, v2):
    if norm(v1) > 0 and norm(v2) > 0:
        return dot(v1, v2) / (norm(v1) * norm(v2))
    else:
        return 0.0

In [51]:
cosine(vecs["猫"], vecs["狗"])

0.97609530509690179

In [52]:
cosine(vecs["猫"], vecs["深圳"])

0.88660692878973246

In [53]:
cosine(vecs["北京"], vecs["深圳"])

0.93054667231075927

In [133]:
def closest(vecs, v, n=10):
    return sorted(vecs.keys(), key=lambda x: cosine(vecs[x], v), reverse=True)[:n]

In [134]:
closest(vecs, vecs["北京"])

['北京', '南京', '上海', '天津', '北平', '成都', '杭州', '西安', '燕京', '沈阳']

In [135]:
closest(vecs, vecs["猫"])

['猫', '狗', '貓', '猪', '叫', '豹', '兔', '疯', '"', '狼']

In [136]:
closest(vecs, vecs["妹妹"])

['妹妹', '哥哥', '姐姐', '弟弟', '姊姊', '丈夫', '妻子', '女兒', '表妹', '姊']

In [137]:
vecs["諳"]

KeyError: '諳'

In [138]:
import jieba

In [139]:
import numpy as np
def summarize(vecs, words):
    found = [vecs[w] for w in words if w in vecs]
    if len(found) > 0:
        return np.mean(found, axis=0)
    else:
        return np.zeros(300)

In [179]:
import re
def sentences_zh(text):
    text = text.replace("\n", "").replace("\u3000", "")
    ss = re.split(r"[。？！“：]+", text)
    return [sent.strip() for sent in ss if len(sent) > 0]

In [180]:
def sentence_space(word_vecs, text):
    space = {}
    sentences = sentences_zh(text)
    for sent in sentences:
        summary = summarize(word_vecs, jieba.cut(sent))
        space[sent] = summary
    return space

"Hands (2.2) - Xiao Hong (1911-1942) was a writer of essays, fiction, and poetry who is widely considered to be China's first feminist novelist. A close friend of Lu Xun, she is best known for her political satire and her depictions of life under Japanese rule and in small towns still governed by feudal customs. Her style, at once sweeping and compassionate, has been compared to that of Tolstoy, Flaubert, and Ba Jin. Published in 1936, this short story--one of her best known--is about a dye-worker's daughter who is ostracized at school because of her darkened hands. Her personal suffering becomes an emblem of her society's assaults on the dignity and integrity of the working class. (View Text | Download Text)" from [here](http://www-personal.umich.edu/~dporter/sampler/sampler.html)

In [181]:
text = open("shou.txt").read()

In [182]:
space = sentence_space(vecs, text)

In [194]:
for s in closest(space, summarize(vecs, jieba.cut("我的狗"))):
    print(s)

从小铁锅煮好的鸡子，我也看到的，是黑的，我以为那是中了毒
我听到床头上有沙沙的声音，好像什么人在我的床头摸索着，我仰过头去，在月光下我看到了是王亚明的黑手，并且把我借给她的那本书放在我的旁边
”她又向角落里走了几步，我看到她的白眼球好像对着我似的
我是叫你下楼，等参观的走了再上来，谁叫你就站在过道呢
我的手是黑的，细看才带点紫色，那两个妹妹也都和我一样
妈的，吃胖了，这里吃的比自家吃的好，是不是
我不要她，我不和她并床……”
爹，你戴着吧，我戴手套本来是没用的
她的虱子，我不挨着她……”
马利亚，真像有这个人一样，她倒在雪地上，我想她没有死吧
